In [1]:
import os

import pandas as pd

from scripts.fasta_processing import read_single_fasta

In [3]:
def create_alignment_files(df_info: pd.DataFrame, phylum: str) -> None:
    cds_cassette_dict = {}

    for row in df_info.iterrows():
        org_name = row[1].org_name
        org_name_protein_id = row[1].org_name_protein_id
        sub_phylum = row[1].sub_phylum
        source = row[1].source
        intron_length_to_stop_codon = row[1].intron_length_to_stop_codon

        match source:
            case "datasets":
                cds_cassette_seq = read_single_fasta(f"../Datasets/{sub_phylum}/{org_name_protein_id}/ncbi_dataset/data/cds_cassette_plain.fa")
            case "psi_blast":
                cds_cassette_seq = read_single_fasta(f"../Sequences_protein_id/{sub_phylum}/{org_name_protein_id.split("__")[1]}/cds_cassette_plain.fa")
            case _:
                raise KeyError(f"Source {source} not recognized")

        # блок для изменения названия org_name для выравнивания
        # первый вариант
        # cds_cassette_dict[f"{sub_phylum}_{org_name}"] = cds_cassette_seq
        # второй вариант
        # cds_cassette_dict[f"{org_name}"] = cds_cassette_seq
        # третий вариант
        cds_cassette_dict[f"{org_name}|{intron_length_to_stop_codon}"] = cds_cassette_seq

    with open(f"../alignment/{phylum}/{phylum}_cds_cassette_motif.fa", "w") as fa_outfile:
        with open(f"../alignment/{phylum}/{phylum}_cds_cassette_motif.aln", "w") as aln_outfile:
            for header, seq in cds_cassette_dict.items():
                fa_outfile.write(f">{header}\n{seq}\n")
                aln_outfile.write(f">{header}\n{seq}\n")

    print(f"alignment files with motif for {phylum} created in ../alignment/{phylum}")

### Множестевнные выравнивания

In [4]:
protostomia = pd.read_csv("../results_summary/protostomia.tsv", sep="\t")
deuterostomia = pd.read_csv("../results_summary/deuterostomia.tsv", sep="\t")

Actinopterygii_list = ["Euteleosteomorpha", "Otomorpha", "Osteoglossomorpha", "Cladistia"]
# создаем таблицу, в которой все найденные виды для лучеперых рыб
df_Actinopterygii_info = deuterostomia[deuterostomia.sub_phylum.isin(Actinopterygii_list)]

# читаем ранее сохраненную в meme suite таблицу
df_Actinopterygii_meme_result = pd.read_csv("../alignment/Actinopterygii/Actinopterygii_cds_cassette_motif.tsv", sep="\t")

# обновляем информационную табличку видами из таблицы с найденными мотивами
df_Actinopterygii_info_upd = df_Actinopterygii_info[df_Actinopterygii_info["org_name"].isin(df_Actinopterygii_meme_result.org_name)]

In [5]:
# создаем файлы для выравнивания видов, у которых нашелся мотив в интроне
create_alignment_files(df_Actinopterygii_info_upd, "Actinopterygii")

Alignment files with motif for Actinopterygii created in ../Alignment/Actinopterygii


In [ ]:
# для созданного файла проводим множественное выравнивание MAFFT 10 итераций в UGENE

### Филогенетический анализ

In [6]:
# нужно запустить эту ячейку, чтобы создалась папка для исследуемой филы
os.makedirs("../iqtree/Actinopterygii_v1", exist_ok=True)
os.makedirs("../iqtree/Actinopterygii_v2", exist_ok=True)
os.makedirs("../iqtree/Actinopterygii_v3", exist_ok=True)

# команда для запуска iqtree (скачана мамбой, окружение то же) через терминал из директории с проектом (Diploma), seed для повторности, -nt 16 ядер
# iqtree -s alignment/Actinopterygii/Actinopterygii_cds_cassette_motif.aln -pre iqtree/Actinopterygii/Actinopterygii -m MFP -bb 1000 -alrt 1000 -seed 12345 -nt 16
# логи лежат в соответствующей папке в файле .log

In [ ]:
# для запуска Fitree нужно сменить окружение: mamba activate figtree
# запускаем просто через figtree в терминал, лучше через обычный терминал, а не в пайчарме
# меняем системную тему на светлую, чтобы было что-то видно в программе
# в программе открываем файл .treefile

# Appearance: line weidth 2
# Tip Labels: font size 15
# какие-то доп. настройки цвета и т.д.

# File -> Export: JPEG